In [1]:
import sys
import os
sys.path.insert(0,os.path.abspath('..'))
from spatial_graphs.AmiraSpatialGraph import AmiraSpatialGraph,MatchBarrels
from spatial_graphs.Landmarks import Landmarks
from spatial_graphs.Surfaces import Surface
from spatial_graphs.Vectors import Vectors
from spatial_graphs.Alignment import Alignment
from dask import compute,multiprocessing,delayed
import pathlib
import shutil
import glob
import pandas as pd
import vtk
from scipy.spatial import distance
import numpy as np

# Create output folders

In [2]:
input_path_spatial_graphs = '/nas1/Data_Mythreya/MotorCortexProject/V10/Registration/Input_Spatial_Graphs/'
surface_resolution = 100
#alignment_types = ['Using_vS1_vM1_Surface_Points',]
alignment_type = 'Using_vS1_vM1_Surface_Points'
input_path_avg_rf = '/nas1/Data_Mythreya/MotorCortexProject/V10/vM1_Ref_Frame/Outputs/Average/'+alignment_type+'/In_SBF_Ref_Frame_With_Scaling_Correction/'
#exp_names = ['MG49_lhs','MG49_rhs','MG50_lhs','MG50_rhs','MG48_lhs','MG48_rhs']
#alignment_types = ['Using_vS1','Using_vM1','Using_Pia_WM_Surfaces','Using_vS1_vM1_Surfaces']


ref_barrels = AmiraSpatialGraph(input_path_avg_rf+'Avg_SBF_Without_Projections.am',generic_graph=True,\
                                barrel_projections_present=False)
ref_pia = Surface(input_path_avg_rf+'Avg_Pia_50.vtk')
ref_wm = Surface(input_path_avg_rf+'Avg_WM_50.vtk')
ref_pia_wm = ref_pia
ref_pia_wm.append(ref_wm.surface)

In [3]:
# create output folder structure
HOME = str(pathlib.Path(input_path_spatial_graphs).parent)
output_root = HOME + '/Outputs/'
pathlib.Path((output_root)).mkdir(exist_ok=True)

output_surfaces = output_root + '1_Surfaces/'
pathlib.Path((output_surfaces)).mkdir(exist_ok=True)

output_axis_field = output_root + '1_Axis_Fields/'
pathlib.Path((output_axis_field)).mkdir(exist_ok=True)

output_stats = output_root + '2_Local_Stats/'
pathlib.Path((output_stats)).mkdir(exist_ok=True)

output_neuron_axis = output_root + '2_Local_Neuron_Axis/'
pathlib.Path((output_neuron_axis)).mkdir(exist_ok=True)

output_alignment_using_surfaces = output_root + '3_Aligned_By_Surfaces/'
pathlib.Path((output_alignment_using_surfaces)).mkdir(exist_ok=True)

output_alignment_using_barrels = output_root + '3_Aligned_By_Barrels/'
pathlib.Path((output_alignment_using_barrels)).mkdir(exist_ok=True)





# Create Pia and WM Surfaces

In [ ]:
@delayed
def create_surface(file,surface_resolution='100'):
    sg = AmiraSpatialGraph(filename=file)
    # flip if required
    #txmat = get_axis_flip_tx_mat(sg)
    #print(txmat)
    #sg.apply_transformation(txmat)
    #sg.write_spatial_graph(root_dst+os.path.basename(file))
    #sg = AmiraSpatialGraph(root_dst+os.path.basename(file))
    #sg_clipped = get_fist_x_z_planes_sg(sg,17)
    #sg_clipped.write_spatial_graph(root_dst+os.path.basename(file)+'_clipped.am')
    os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
              format(file,output_surfaces+os.path.basename(file)[:-3],str(100)))
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
    
    if pia.surface.GetNumberOfCells()>0 and wm.surface.GetNumberOfCells()>0:
        pia.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
        wm.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
        
        os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
              format(file,output_surfaces+os.path.basename(file)[:-3],str(450)))
        pia_450 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
        pia_450.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open_450.vtk')
        
        wm_450 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
        wm_450.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open_450.vtk')
            
    else:
        # create a delunay surface as the traditional way has failed
        pia = Surface(pts=sg.pia.edge_pt_coords)
        wm = Surface(pts=sg.wm.edge_pt_coords)
        pia_hull = pia.create_delunay_surface_3d(return_hull=True,output_filename=output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
        wm_hull = wm.create_delunay_surface_3d(return_hull=True,output_filename=output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
        
        # clip at the bottom
        shutil.copyfile(src=file,dst=output_surfaces+os.path.basename(file)[:-3]+'_barrels.am')
        pia.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
        wm.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
        #wm.create_axis_field(pia,op_sg_name=output_surfaces+os.path.basename(file)[:-3]+'_axis_field.am',flip_normals=False,\
        #                              return_axes=True)

In [ ]:
fn_list_surfs = []
file_names = sorted(glob.glob(input_path_spatial_graphs+'*.am'))
for i in range(len(file_names)):
    #if os.path.basename(file_names[i])=='RA20150804_2_4x-_for_cell_merge_.am_Segmented_corrected_DONE.am':
    fn_list_surfs.append(create_surface(file_names[i]))

compute(fn_list_surfs)

# Generate Axis Field

In [ ]:
# Create axis with the smooth wm (voxel size 450) 
file_names = sorted(glob.glob(input_path_spatial_graphs+'*.am'))
for i in range(len(file_names)):
    file = file_names[i]
    #if os.path.basename(file)[:-3] == 'RA20150804_2_4x-_for_cell_merge_.am_Segmented_corrected_DONE':
    pia_100 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm_100 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    wm_450 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open_450.vtk')

    if wm_450.surface.GetNumberOfCells() > 0:
        wm_to_use = wm_450
    else:
        wm_to_use = wm_100
    
    flipped_axes = wm_to_use.create_axis_field(pia_100,op_sg_name=None,flip_normals=True,return_axes=True,\
                                            max_allowed_axis_length=5000,check_source_double_touch=True)
    axes = wm_to_use.create_axis_field(pia_100,op_sg_name=None,flip_normals=False,return_axes=True,\
                                    max_allowed_axis_length=5000,check_source_double_touch=True)
    sg = AmiraSpatialGraph(generic_graph=True)
    for edge in flipped_axes:
        sg.graph_data.add_edge(edge[0],edge[1])
    for edge in axes:
        sg.graph_data.add_edge(edge[0],edge[1])
    #sg.write_spatial_graph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am')

    # trim the axis field as per the pia and wm surfaces of 100 micron voxel resolution
    selected_axis_field_sg,wm_pts,pia_pts,missing_axis_field = pia_100.get_axis_field_within_instersecting_surface_with_trimming\
                                (pia_100.surface,wm_100.surface,sg.graph_data.edge_list,vec_extension_offset=0.75)
    selected_axis_field_sg.write_spatial_graph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am')

        
        

# Stats of input neuron and surfaces

In [ ]:
def write_edge(edge,filename):
    axis_sg = AmiraSpatialGraph(generic_graph=True)
    axis_sg.graph_data.add_edge(edge[0],edge[1])
    axis_sg.write_spatial_graph(filename)

    return axis_sg

    

In [ ]:
def resample_edge(edge,num_pts_to_be_inserted):
    resampled_edge = []
    #resample_edge.append(edge[0])
    n = np.array(edge[len(edge)-1]) - np.array(edge[0])
    n_unit = n / np.linalg.norm(n)
    edge_len = Vectors().get_vec_length(edge)
    for i in range(num_pts_to_be_inserted):
        resampled_edge.append(i*n_unit*edge_len/num_pts_to_be_inserted+edge[0])
    return edge

In [ ]:
def get_pca_direction(apical_dendrite_pts,res=1):
#     resampled_pts = []
#     for edge in apical_dendrite_edges:
#         for i in range(len(edge)-1):
#             length = distance.euclidean(edge[i],edge[i+1])
#             if length > res:
#                 num_pts_to_be_inserted = int(length/res)
#                 resampled_edge = resample_edge(edge,num_pts_to_be_inserted)
#                 for i in range(len(resampled_edge)):
#                     resampled_pts.append(resampled_edge[i])
    
    pca = PCA(n_components=3,)
    transformed_pca = pca.fit_transform(X=apical_dendrite_pts)
    #print(transformed_pca)
    return pca.components_

In [ ]:
def get_wm_intersection_pt(wm,soma_pt,pia_pt):
    wm_pt = []
    for j in range(500,6000,50):
        opposing_pt = Vectors().create_pt_along_vector_at_given_distance(-j,soma_pt,pia_pt)
        #print(opposing_pt)
        #Landmarks(pts=[opposing_pt]).write_landmarks(output_root+'oppsing_pt')
        wm_pt,dist = wm.get_vector_intersection_pt(soma_pt,opposing_pt,extrapolation_len=0)
        #Landmarks(pts=[opposing_pt]).write_landmarks(output_root+'wm_pt')
        #print(wm_pt)
        if len(wm_pt)>0:
            break
    return wm_pt

In [ ]:
cols = ['Exp_Name','Pia_Area','WM_Area','Pia_Volume','WM_Volume',\
        'Cortical_Thickness_Mean','Cortical_Thickness_Std','Cortical_Thickness_CoV',\
        'Cortical_Thickness_vS1_Mean','Cortical_Thickness_vS1_Std','Cortical_Thickness_vS1_CoV',]
df_main = pd.DataFrame(columns=cols)

for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    print(file)
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    pia_closed = pia.create_delunay_surface_3d(return_hull=True,make_cube=False)
    wm_closed = wm.create_delunay_surface_3d(return_hull=True,make_cube=False)
    axis_field = AmiraSpatialGraph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am',generic_graph=True)
    
    sg = AmiraSpatialGraph(file)
    vs1_sg = AmiraSpatialGraph(generic_graph=True)
    # create hull around s1 and get axis field for s1
    for pt in sg.barrels.all_rows_graphdata.edge_pt_coords:
        min_edge,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = \
        Vectors().get_nearest_axis_to_pt(pt,axis_field.graph_data.edge_list,axis_validation_distance=3000)
        vs1_sg.graph_data.add_edge(min_edge[0],min_edge[1])
    vs1_sg.write_spatial_graph(output_axis_field+os.path.basename(file)[:-3]+'_vS1_Axis_Field.am')
    
    # find vector lengths for whole surface and vs1 area
    vec_lens,bla1,bla2 = Vectors().get_vec_lengths(axis_field.graph_data.edge_list)
    vec_lens_vs1,bla,blee = Vectors().get_vec_lengths(vs1_sg.graph_data.edge_list)
        
    df = pd.DataFrame(columns=df_main.columns)
    df['Exp_Name'] = [os.path.basename(file)[:-3]]
    df['Pia_Area'] = [pia.get_surface_properties(prop='area')]
    df['WM_Area'] = [wm.get_surface_properties(prop='area')]
    df['Pia_Volume'] = [Surface(polydata=pia_closed).get_surface_properties(prop='volume')]
    df['WM_Volume'] = [Surface(polydata=wm_closed).get_surface_properties(prop='volume')]
    df['Cortical_Thickness_Mean'] = [np.array(vec_lens).mean(axis=0)]
    df['Cortical_Thickness_Std'] = [np.array(vec_lens).std(axis=0)]
    df['Cortical_Thickness_CoV'] = df['Cortical_Thickness_Std'] / df['Cortical_Thickness_Mean'] * 100
    df['Cortical_Thickness_vS1_Mean'] = [np.array(vec_lens_vs1).mean(axis=0)]
    df['Cortical_Thickness_vS1_Std'] = [np.array(vec_lens_vs1).std(axis=0)]
    df['Cortical_Thickness_vS1_CoV'] = df['Cortical_Thickness_vS1_Std'] / df['Cortical_Thickness_vS1_Mean'] * 100
        
    df_main = df_main.append(df)

df_main.to_csv(output_stats+'Surface_Area_and_Thickness.csv')
    

In [ ]:
df_main

In [ ]:
cols = ['Exp_Name','Pia_Area','WM_Area','Volume','Cortical_Thickness_Mean','Cortical_Thickness_Std','Cortical_Thickness_CoV',\
        'Apical_Dendrite_Present','WM_Present_Below_Neuron','Neuron_Depth_from_Pia','Neuron_Height_from_WM','Neuron_Axis_Length',\
        'Nearest_Axis_Depth_from_Pia','Nearest_Axis_Height_from_WM','Nearest_Axis_Length',\
        'Angle_to_Nearest_Axis',]
df_main = pd.DataFrame(columns=cols)
for file in sorted(glob.glob(output_corrected_spatial_graphs+'*.am')):
    #if os.path.basename(file)[:-3] == 'RA20160401_Cell_B_DONE':
    print(os.path.basename(file)[:-3])
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    axis_field = AmiraSpatialGraph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am',generic_graph=True)
    
    sg = AmiraSpatialGraph(file)
    soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
    Landmarks(pts=[soma_center]).write_landmarks(output_neuron_axis+os.path.basename(file)[:-3]+'_Soma_Centroid.landmarksAscii')

    dendrite_pts = []    
    for pt in sg.neuron.dendrite.apical_dendrite.edge_pt_coords:
        dendrite_pts.append(pt)
    for pt in sg.neuron.dendrite.basal_dendrite.edge_pt_coords:
        dendrite_pts.append(pt)   
    dendrite_mean = np.array(dendrite_pts).mean(axis=0)

    # write dendrite for easy visualization
    sg.write_sub_spatial_graph(sg.neuron.dendrite.apical_dendrite,output_neuron_axis+os.path.basename(file)[:-3]+\
                               '_apical_dendrite.am')
    sg.write_sub_spatial_graph(sg.neuron.dendrite.basal_dendrite,output_neuron_axis+os.path.basename(file)[:-3]+\
                               '_basal_dendrite.am')

    # Avg Local Axis 
    nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(axis_field.graph_data.edge_list,[soma_center],800)
    nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
    unit_vecs = []
    for axis in nearby_axes:
        nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
        unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
    nearby_axes_sg.write_spatial_graph(output_neuron_axis+os.path.basename(file)[:-3]+'_axis_field_selected.am')
    avg_uv = np.array(unit_vecs).mean(axis=0)
    end_pt = (avg_uv*6000 + soma_center)
    pia_proj_pt_neuron_using_local_avg_axis,soma_depth_using_local_avg = pia.get_vector_intersection_pt(end_pt,soma_center,extrapolation_len=0)
    write_edge([soma_center,pia_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_axis_local_avg.am')
    
    wm_proj_pt_neuron_using_local_avg_axis = get_wm_intersection_pt(wm,soma_center,pia_proj_pt_neuron_using_local_avg_axis)
    if len(wm_proj_pt_neuron_using_local_avg_axis) > 0:
        write_edge([soma_center,wm_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_wm_axis_local_avg.am')
    else:
        print('no wm intersection found using avg local axis for {}'.format(os.path.basename(file)))
    soma_depth_using_local_avg = Vectors().get_vec_length([soma_center,pia_proj_pt_neuron_using_local_avg_axis])
    # decide which method of depth measurement to use
    # using the apical dendrite axis works best for L5 and below neurons
    # where as local axis is good for L2/3
    print(soma_depth_using_local_avg)

    if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords) > 0:
        # Get apical dend axis using pca
        pca_components = get_pca_direction(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)
        #Landmarks(pts=resampled_pts).write_landmarks(output_root+'pts')
        end_pt = Vectors().create_pt_along_vector_at_given_distance(6000,[0,0,0],pca_components[0,:])
        #Landmarks(pts=[end_pt+soma_center]).write_landmarks(output_root+'end_pt')
        #print(end_pt)
        angle = Vectors().get_angle_between_vectors([soma_center,pia_proj_pt_neuron_using_local_avg_axis],\
                         [soma_center,end_pt+soma_center],ignore_opposite_direction=False)
        print(angle)
        pia_proj_pt_neuron = []
        if angle < 90:
            # pca direction is correct.. project to pia
            end_pt = Vectors().create_pt_along_vector_at_given_distance(6000,[0,0,0],pca_components[0,:])
            print(end_pt)
            Landmarks(pts=[end_pt+soma_center]).write_landmarks(output_root+'end_pt')
            pia_proj_pt_neuron,soma_depth  = pia.get_vector_intersection_pt(end_pt+soma_center,soma_center,extrapolation_len=0)
            write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_pca_axis.am')
        else:
            # the pca is inverted, so straighten it
            #print('opposite')
            end_pt = Vectors().create_pt_along_vector_at_given_distance(-6000,[0,0,0],pca_components[0,:])
            pia_proj_pt_neuron,soma_depth = pia.get_vector_intersection_pt(end_pt+soma_center,soma_center,extrapolation_len=0)
            write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_pca_axis.am')
        
        wm_proj_pt_neuron = get_wm_intersection_pt(wm,soma_center,pia_proj_pt_neuron)
        if len (wm_proj_pt_neuron) > 0:
            write_edge([soma_center,wm_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_wm_pca_axis.am')
        else:
            print('no wm intersection found using pca for {}'.format(os.path.basename(file)))
            
    # select the neron axis based on whether the depth of the neuron somata
    # for upper layer cells use avg axis, as they generally dont have enough apical dend
    # otherwise use dend pca based axis
    if soma_depth_using_local_avg < 500 or len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords) == 0\
        or os.path.basename(file)[:-3] == 'RA20160401_Cell_B_DONE':
        # this is a inverted cell... so do not invert the axis.. lets use the avg axis for this one:
        write_edge([soma_center,pia_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_axis.am')
        if len(wm_proj_pt_neuron_using_local_avg_axis) > 0:
            write_edge([soma_center,wm_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_wm_axis.am')
    else:
        write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_pia_axis.am')
        if len(wm_proj_pt_neuron) > 0:
            write_edge([soma_center,wm_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_soma_wm_axis.am')

In [ ]:
cols = ['Exp_Name','Pia_Area','WM_Area','Volume','Cortical_Thickness_Mean','Cortical_Thickness_Std','Cortical_Thickness_CoV',\
        'Apical_Dendrite_Present','WM_Present_Below_Neuron','Neuron_Depth_from_Pia','Neuron_Height_from_WM','Neuron_Axis_Length',\
        'Nearest_Axis_Depth_from_Pia','Nearest_Axis_Height_from_WM','Nearest_Axis_Length',\
        'Angle_to_Nearest_Axis','Neuron_Sticking_OutOf_Pia',]
df_main = pd.DataFrame(columns=cols)
start = 0
end= 10
for i in range(len(sorted(glob.glob(output_surfaces+'*_pia_bottom_open.vtk')))):
    if i < start or i >= end:
        continue
    df = pd.DataFrame(columns=df_main.columns)
    pia_file = sorted(glob.glob(output_surfaces+'*_pia_bottom_open.vtk'))[i]
    exp_name = os.path.basename(pia_file)[0:-20]
#     if exp_name != 'RA20130626_2':
#         continue
    print(exp_name)
    sg = AmiraSpatialGraph(output_surfaces+exp_name+'_barrels.am')
    pia_closed = Surface(output_surfaces+exp_name+'_pia.vtk')
    wm_closed = Surface(output_surfaces+exp_name+'_WM.vtk')
    pia = Surface(output_surfaces+os.path.basename(pia_file)[0:-20]+'_pia_bottom_open.vtk')
    wm = Surface(output_surfaces+os.path.basename(pia_file)[0:-20]+'_WM_bottom_open.vtk')
    
    axis_field = wm.create_axis_field(pia,op_sg_name=output_surfaces+exp_name+'_axis_field.am',flip_normals=False,return_axes=True)
    
    df['Exp_Name'] = [exp_name]
    df['Pia_Area'] = [pia.get_surface_properties(prop='area')]
    df['WM_Area'] = [wm.get_surface_properties(prop='area')]
    df['Volume'] = [pia_closed.get_surface_properties(prop='volume')]
    vec_lens = []
    for edge in axis_field:
        vec_lens.append(Vectors().get_vec_length(edge))
    df['Cortical_Thickness_Mean'] = [np.array(vec_lens).mean(axis=0)]
    df['Cortical_Thickness_Std'] = [np.array(vec_lens).std(axis=0)]
    df['Cortical_Thickness_CoV'] = [100*np.array(vec_lens).std(axis=0)/np.array(vec_lens).mean(axis=0)]
    
    # find neuron axis, nearest axis and angle between them
    soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
    apical_dend_center = np.array(sg.neuron.dendrite.apical_dendrite.edge_pt_coords).mean(axis=0)
    if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords) == 0:
        df['Apical_Dendrite_Present'] = ['False']
        df_main = df_main.append(df)
        continue
    #Landmarks(pts=[soma_center,apical_dend_center]).write_landmarks(output_surfaces+'a')
    pia_proj_pt_neuron,soma_depth = pia.get_vector_intersection_pt(soma_center,apical_dend_center,extrapolation_len=10000)
    wm_proj_pt_neuron,soma_height = wm.get_vector_intersection_pt(soma_center,apical_dend_center,extrapolation_len=-10000)
    print(wm_proj_pt_neuron,pia_proj_pt_neuron)
    if len(wm_proj_pt_neuron) == 0 or len(pia_proj_pt_neuron)==0:
        df['WM_Present_Below_Neuron'] = ['False']
        df_main = df_main.append(df)
        continue
    write_edge([wm_proj_pt_neuron,pia_proj_pt_neuron],output_surfaces+exp_name+'_neuron_axis.am')
    
    nearest_axis,min_dist, min_dist_from_wm, min_dist_from_pia, pt_on_vector = Vectors().\
                        get_nearest_axis_to_pt(soma_center,axis_field)
    write_edge(nearest_axis,output_surfaces+exp_name+'_nearest_axis.am')
    
    angle = Vectors().get_angle_between_vectors([wm_proj_pt_neuron,pia_proj_pt_neuron],nearest_axis)
    
    df['Apical_Dendrite_Present'] = ['True']
    df['WM_Present_Below_Neuron'] = ['True']
    df['Neuron_Depth_from_Pia'] = [soma_depth]
    df['Neuron_Height_from_WM'] = [soma_height]
    df['Neuron_Axis_Length'] = [soma_depth+soma_height]
    df['Nearest_Axis_Depth_from_Pia'] = [min_dist_from_pia]
    df['Nearest_Axis_Height_from_WM'] = [min_dist_from_wm]
    df['Nearest_Axis_Length'] = [min_dist_from_pia+min_dist_from_wm]
    df['Angle_to_Nearest_Axis'] = [angle]
    is_sticking_out = pia.is_neuron_sticking_out(sg)
    df['Neuron_Sticking_OutOf_Pia'] = [is_sticking_out]
    if not (is_sticking_out):
        # write this valid exp for registration
        sg.write_spatial_graph(output_valid+exp_name+'.am')
    df_main = df_main.append(df)

df_main.to_csv(output_root+'1.Input_Neuron_Stats_{}.csv'.format(end))

In [ ]:
df_main = pd.DataFrame(columns=cols)
for csv in sorted(glob.glob(output_root+'/*.csv')):
    df = pd.read_csv(csv,)
    df_main = df_main.append(df,)

In [ ]:
df_main

In [ ]:
df_main.to_csv(output_root+'1.Neuron_Stats.csv')

# Register Using Surfaces

In [4]:
# 

In [5]:
i = 0
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    print(i, os.path.basename(file))
    i = i + 1
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
    
    pia_wm = pia
    pia_wm.append(wm.surface)

    aligner = Alignment(ref_pia_wm.surface,pia_wm.surface)
    txmat = aligner.get_transformation_matrix()
    icp = aligner.get_icp_transform()
    
    aligner = Alignment()
    
    Alignment().transform_folders(output_root,output_alignment_using_surfaces,os.path.basename(file)[:-3],\
                            icp=icp,txmat = txmat,)

0 RA20130319_2_DONE.am
1 RA20130320_corrected_DONE.am
2 RA20130402_corrected_2_wm_countours_corrected_shifted_and_merged_DONE.am
3 RA20130404_1_DONE.am
4 RA20130404_2_corrected_DONE.am
5 RA20130507_1_corrected_shifted_and_merged_DONE_feature_corrected.am
6 RA20130507_2_corrected_DONE.am
7 RA20130508_1_corrected_shifted_and_merged_DONE.am
8 RA20130513_corrected-extended_2corrected_shifted_and_merged_DONE.am
9 RA20130603_DONE.am
10 RA20130605_1_DONE.am
11 RA20130606_DONE.am
12 RA20130607_1_DONE_feature_corrected_corrected.am
13 RA20130607_2_DONE.am
14 RA20130626_1_corrected_shifted_and_merged_DONE_feature_corrected.am
15 RA20130626_2_DONE.am
16 RA20130627_Cell_2_DONE.am
17 RA20130627_Cell_3_DONE.am
18 RA20130627_DONE.am
19 RA20130628_2_Cell_1_corrected_shifted_and_merged_DONE.am
20 RA20130628_2_Cell_2_corrected_shifted_and_merged_DONE.am
21 RA20130628_2_Cell_3_corrected_shifted_and_merged_DONE_feature_corrected_corrected.am
22 RA20130628_shifted_and_merged_DONE.am
23 RA20130701_2_DONE.am

# Register Using Barrels

In [ ]:
for file in sorted(glob.glob(input_path_spatial_graphs+'*.am')):
    sg = AmiraSpatialGraph(file,barrel_projections_present=False)
    
    if len(sg.barrels.all_rows_graphdata.edge_list) > 0:
        matching_barrels = MatchBarrels(ref_barrels.barrels,sg.barrels,use_barrels_alone=True)
        ref_s1_pts,s1_pts = matching_barrels.get_matching_barrel_centroids()
        landmarks = Landmarks(pts=s1_pts)
        icp,txmat = landmarks.align_landmarks(ref_s1_pts)
        aligner = Alignment()
        #print(txmat)
        #print(txmat)
        Alignment().transform_folders(output_root,output_alignment_using_barrels,os.path.basename(file)[:-3],\
                                icp=icp,txmat = txmat,)


In [ ]:
os.path.basename(file)[:-3]

In [ ]:
os.path.basename('1_Axis_Fields').find('Align')

In [ ]:
def get_number_of_axes_from_field(axis_field,number_of_axes_needed,neuron_axis=[]):
    axis_filed_len = len(axis_field)
    offset = int(axis_filed_len/number_of_axes_needed)
    return_list = []
    if len(neuron_axis) > 0:
        if neuron_axis is not None:
            for i in range(len(axis_field)):
                return_list.append(axis_field[i])
                i = i + offset
                if len(return_list) == number_of_axes_needed:
                    return return_list
        
    else:
        
        for i in range(len(axis_field)):
            return_list.append(axis_field[i])
            i = i + offset
            if len(return_list) == number_of_axes_needed:
                return return_list

In [ ]:
ref_path = '/nas1/Data_Mythreya/MotorCortexProject/V8/vM1_Ref_Frame/Outputs_200/Average/Using_vS1_vM1_Surface_Points/'
ref_barrels_sg_file = '/nas1/Data_Mythreya/MotorCortexProject/V8/vM1_Ref_Frame/Outputs_200/Alignment/Using_vS1_vM1_Surface_Points/Spatial_Graphs/MG49_lhs_sbf.am'   
ref_barrels_sg = AmiraSpatialGraph(ref_barrels_sg_file,barrel_projections_present=True)
ref_pia = Surface(ref_path + 'Avg_Pia.vtk')
ref_wm = Surface(ref_path + 'Avg_WM.vtk')
ref_axis_field = AmiraSpatialGraph(ref_path + 'Avg_vM1_Axis_Field.am',generic_graph=True)

ref_surf = ref_pia
ref_surf.append(ref_wm.surface)

In [ ]:
# surface alignment 
for i in range(len(sorted(glob.glob(output_valid+'/*.am')))):
    file = sorted(glob.glob(output_valid+'/*.am'))[i]
    sg = AmiraSpatialGraph(file,barrel_projections_present=True)
    pia =  Surface(output_surfaces+os.path.basename(file)[0:-3]+'_pia_bottom_open.vtk')
    wm =  Surface(output_surfaces+os.path.basename(file)[0:-3]+'_WM_bottom_open.vtk') 

    surf = pia
    surf.append(wm.surface)

    aligner = Alignment(ref_surf.surface,surf.surface)
    txmat = aligner.get_transformation_matrix()
    icp = aligner.get_icp_transform()
    
    sg.apply_transformation(txmat)
    sg.write_spatial_graph(output_alignment_using_surfaces+os.path.basename(file))
    surf.apply_icp_transform(icp)
    surf.write_surface_mesh(output_alignment_using_surfaces+os.path.basename(file)[0:-3]+'.vtk')
    

In [ ]:
all_neurons = AmiraSpatialGraph(generic_graph=True)
for neuron_file in glob.glob(output_alignment_using_surfaces+'/*.am'):
    sg_neuron = AmiraSpatialGraph(neuron_file)
    all_neurons.graph_data = all_neurons.combine_subgraphs([all_neurons.graph_data,sg_neuron.neuron.all_neurites_subgraphdata])

    
all_neurons.write_spatial_graph(output_root+'All_Neurons_Registered_To_vM1.am')

# with Scaling

In [ ]:
df = pd.read_csv(output_root+'1.Neuron_Stats.csv')

In [ ]:
df[df['WM_Present_Below_Neuron']==True]['Pia_Area'].mean()

In [ ]:
ref_pia.get_surface_properties(prop='area')
#ref_wm.get_surface_properties(prop='area')/ df['WM_Area'].mean()


In [ ]:
df['Pia_Area'].mean()

In [ ]:
# surface alignment scaling
# 
for i in range(len(sorted(glob.glob(output_valid+'/*.am')))):
    file = sorted(glob.glob(output_valid+'/*.am'))[i]
    sg = AmiraSpatialGraph(file,barrel_projections_present=True)
    pia =  Surface(output_surfaces+os.path.basename(file)[0:-3]+'_pia_bottom_open.vtk')
    wm =  Surface(output_surfaces+os.path.basename(file)[0:-3]+'_WM_bottom_open.vtk') 

    surf = pia
    surf.append(wm.surface)

    aligner = Alignment(ref_surf.surface,surf.surface,mode='similarity')
    txmat = aligner.get_transformation_matrix()
    icp = aligner.get_icp_transform()
    
    sg.apply_transformation(txmat)
    sg.write_spatial_graph(output_alignment_using_surfaces_scaling+os.path.basename(file))
    surf.apply_icp_transform(icp)
    surf.write_surface_mesh(output_alignment_using_surfaces_scaling+os.path.basename(file)[0:-3]+'.vtk')

In [ ]:
# read valid neuron graphs
# align using barrels and axis around neuron
for i in range(len(sorted(glob.glob(output_valid+'/*.am')))):
    file = sorted(glob.glob(output_valid+'/*.am'))[i]
    sg = AmiraSpatialGraph(file,barrel_projections_present=True)
    pia =  Surface(output_surfaces+os.path.basename(file)[0:-3]+'_pia_bottom_open.vtk')
    wm =  Surface(output_surfaces+os.path.basename(file)[0:-3]+'_WM_bottom_open.vtk') 
    axis_field = AmiraSpatialGraph(output_surfaces+os.path.basename(file)[0:-3]+'_axis_field.am',generic_graph=True)
    nearest_axis = AmiraSpatialGraph(output_surfaces+os.path.basename(file)[0:-3]+'_nearest_axis.am',generic_graph=True)
    matching_barrels = MatchBarrels(ref_barrels_sg.barrels,sg.barrels,use_projections_alone=True)
    ref_s1_pts,s1_pts = matching_barrels.get_matching_barrel_centroids()

    selected_ref_axes = get_number_of_axes_from_field(ref_axis_field.graph_data.edge_list,len(ref_s1_pts))
    selected_axes = get_number_of_axes_from_field(axis_field.graph_data.edge_list,len(s1_pts))

    ref_selected_sg = AmiraSpatialGraph(generic_graph=True)
    selected_sg = AmiraSpatialGraph(generic_graph=True)

    print(len(selected_ref_axes))
    ref_pts = []
    pts = []
    for pt in ref_s1_pts:
        ref_pts.append(pt)
    for axis in selected_ref_axes:
        ref_pts.append(axis[0])
        ref_pts.append(axis[1])
        ref_selected_sg.graph_data.add_edge(axis[0],axis[1])
    for pt in s1_pts:
        pts.append(pt)
    for axis in selected_axes:
        pts.append(axis[0])
        pts.append(axis[1])
        selected_sg.graph_data.add_edge(axis[0],axis[1])
    print(len(ref_pts),len(pts))
    ref_selected_sg.write_spatial_graph(output_path_alignment+alignment_type+'/'+ref_exp+'ref_axes.am')
    selected_sg.write_spatial_graph(output_path_alignment+alignment_type+'/'+exp+'axes.am')

    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)
    
    sg.apply_transformation(txmat)
    sg.write_spatial_graph(output_root+os.path.basename(file))
    break

In [ ]:
def vm1_local_alignment(sg,pia,ref_pia,number_of_pts=1,ref_pca = []):
    # locally align vm1 surface with the reference vm1 surface
    # by taking ref pts that intersect the respictive pia along an axis
    # axis 1 : vector along the neuron (apical dendrite direction)
    # axis 2 : pca 2 of ref vm1
    ref_pts= []
    pts = []
    soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
    if (len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)) > 0:                         
        apical_dend_center = np.array(sg.neuron.dendrite.apical_dendrite.edge_pt_coords).mean(axis=0)
    else: # in case there is only basal dend
        apical_dend_center = np.array(sg.neuron.dendrite.basal_dendrite.edge_pt_coords).mean(axis=0)

    pia_proj_pt_neuron,depth = pia.get_vector_intersection_pt(soma_center,apical_dend_center,extrapolation_len=10000)

    ref_pia_proj_pt_neuron,ref_depth = ref_pia.get_vector_intersection_pt(soma_center,apical_dend_center,\
                                                                        extrapolation_len=10000)
    #print(ref_pia_proj_pt_neuron,pia_proj_pt_neuron)
    if len(ref_pia_proj_pt_neuron) == 0 or len(pia_proj_pt_neuron)==0:
        return [],[]
    ref_pts.append(ref_pia_proj_pt_neuron)
    pts.append(pia_proj_pt_neuron)

    if number_of_pts ==2:
        pia_proj_pt_center,depth = pia.get_vector_intersection_pt(ref_pca.graph_data.edge_list[0][0],\
                                                           ref_pca.graph_data.edge_list[0][1],\
                                                           extrapolation_len=-10000)
        ref_pia_proj_pt_center,ref_depth = ref_pia.get_vector_intersection_pt(ref_pca.graph_data.edge_list[0][0],\
                                                           ref_pca.graph_data.edge_list[0][1],\
                                                           extrapolation_len=-10000)

        #print(ref_pia_proj_pt_center,pia_proj_pt_center)
        if len(ref_pia_proj_pt_center) == 0 or len(pia_proj_pt_center)==0:
            return [],[]
        ref_pts.append(ref_pia_proj_pt_center)
        pts.append(pia_proj_pt_center)

    print(ref_pts,pts)
    if len(ref_pts) != len(pts) or len(ref_pts)==0 or len(pts)==0:
        return [],[]
    landmarks = Landmarks(pts=pts)
    icp,txmat = landmarks.align_landmarks(ref_pts)
    
    return icp,txmat

In [ ]:
def get_soma_to_apical_center_distance(sg,):
    soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
    if (len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)) > 0:                         
        apical_dend_center = np.array(sg.neuron.dendrite.apical_dendrite.edge_pt_coords).mean(axis=0)
    else: # in case there is only basal dend
        apical_dend_center = np.array(sg.neuron.dendrite.basal_dendrite.edge_pt_coords).mean(axis=0)
    
    return distance.euclidean(soma_center,apical_dend_center)

In [ ]:
def get_soma_and_apical_centers(sg):
    soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
    if (len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)) > 0:                         
        apical_dend_center = np.array(sg.neuron.dendrite.apical_dendrite.edge_pt_coords).mean(axis=0)
    else: # in case there is only basal dend
        apical_dend_center = np.array(sg.neuron.dendrite.basal_dendrite.edge_pt_coords).mean(axis=0)
    return soma_center[0:3],apical_dend_center[0:3]

In [ ]:
def get_ref_and_neuron_axes(sg,pia,wm,ref_pia,ref_wm,ref_pca = []):
    # locally align vm1 surface with the reference vm1 surface
    # by taking ref pts that intersect the respictive pia along an axis
    # axis 1 : vector along the neuron (apical dendrite direction)
    # axis 2 : pca 2 of ref vm1
    ref_pts= []
    pts = []
    soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
    if (len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)) > 0:                         
        apical_dend_center = np.array(sg.neuron.dendrite.apical_dendrite.edge_pt_coords).mean(axis=0)
    else: # in case there is only basal dend
        apical_dend_center = np.array(sg.neuron.dendrite.basal_dendrite.edge_pt_coords).mean(axis=0)

    pia_proj_pt,depth = ref_pia.get_vector_intersection_pt(soma_center,apical_dend_center,extrapolation_len=10000)

    ref_pia_proj_pt,ref_depth = ref_pia.get_vector_intersection_pt(ref_pca.graph_data.edge_list[0][0],\
                                                       ref_pca.graph_data.edge_list[0][1],\
                                                       extrapolation_len=-10000)
    #print(ref_pia_proj_pt_neuron,pia_proj_pt_neuron)
    ref_pts.append(ref_pia_proj_pt)
    pts.append(pia_proj_pt)

    
    wm_proj_pt,depth = ref_wm.get_vector_intersection_pt(soma_center,apical_dend_center,extrapolation_len=-10000)
    wm_ref_proj_pt,ref_depth = ref_wm.get_vector_intersection_pt(ref_pca.graph_data.edge_list[0][0],\
                                                       ref_pca.graph_data.edge_list[0][1],\
                                                       extrapolation_len=10000)

    #print(ref_pia_proj_pt_center,pia_proj_pt_center)
    ref_pts.append(wm_ref_proj_pt)
    pts.append(wm_proj_pt)

    #landmarks = Landmarks(pts=pts)
    #icp,txmat = landmarks.align_landmarks(ref_pts)
    
    return ref_pts,pts

In [ ]:
def push_back_sticking_out_neuron(pia,sg,output_filename=None,step_size=10):
    if (pia.is_neuron_sticking_out(sg)):
        soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
        #print('found intersection')
        if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords) > 0:
            apical_center = np.array(sg.neuron.dendrite.apical_dendrite.edge_pt_coords).mean(axis=0)
        else:
            apical_center = np.array(sg.neuron.dendrite.basal_dendrite.edge_pt_coords).mean(axis=0)
        new_soma_center = pia.create_pt_along_vector_at_given_distance(-step_size,soma_center,apical_center)
        translation = soma_center - new_soma_center
        txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, -translation[0],-translation[1],-translation[2],1]
        sg.apply_transformation_to_labelled_subgraph(sg.neuron.all_neurites_subgraphdata,np.reshape(txmat,[4,4]))
        sg.update_graph_data()
    if output_filename is not None:
        sg.write_spatial_graph(output_filename)
    return sg

In [ ]:
# Get reference surface 
alignment_type = 'Using_vS1_vM1_Surface_Points'
ref_pia = Surface(filename = input_path_avg_rf+alignment_type+'/Avg_Pia.vtk')
ref_wm = Surface(filename = input_path_avg_rf+alignment_type+'/Avg_WM.vtk')
ref_pia_wm = Surface(filename = input_path_avg_rf+alignment_type+'/Avg_Pia.vtk')
#ref_pia_wm.append(ref_wm.surface)

ref_vM1 = Surface(filename = input_path_avg_rf+alignment_type+'/Avg_vM1.vtk')
ref_vM1_pia = Surface(filename = input_path_avg_rf+alignment_type+'/Avg_vM1_Pia.vtk')
ref_vM1_wm = Surface(filename = input_path_avg_rf+alignment_type+'/Avg_vM1_WM.vtk')
ref_vM1_pia_wm = Surface(filename = input_path_avg_rf+'/'+alignment_type+'/'+'Avg_vM1_Pia.vtk')
ref_vM1_pia_wm.append(ref_vM1_wm.surface)

ref_barrels_sg = AmiraSpatialGraph(input_path_avg_rf+alignment_type+'/sbf_without_projections.am',\
                                   barrel_projections_present=False)

ref_pca_0 = AmiraSpatialGraph(input_path_avg_rf+alignment_type+'/Avg_vM1_Pca_0',generic_graph=True)
ref_pca_1 = AmiraSpatialGraph(input_path_avg_rf+alignment_type+'/Avg_vM1_Pca_1',generic_graph=True)
ref_pca_2 = AmiraSpatialGraph(input_path_avg_rf+alignment_type+'/Avg_vM1_Pca_2',generic_graph=True)

In [ ]:
#@delayed
def register_neurons(sg_filename,alignment_type,check_neuron_sticking_out=False):
    

    # make surface
    pia = Surface(filename = output_surfaces+sg_filename+'_pia_bottom_open.vtk')
    wm = Surface(filename = output_surfaces+sg_filename+'_WM_bottom_open.vtk')
    pia_wm = Surface(polydata = pia.surface)
    #pia_wm.append(wm.surface)
    
    sg = AmiraSpatialGraph(filename=output_corrected_spatial_graphs+sg_filename+'.am',\
                           barrel_projections_present=False)
    
    ################ Step 0###################################
    # check if neuron is sticking put of pia, if so push it back
    ##########################################################
    if check_neuron_sticking_out:
        sg = push_back_sticking_out_neuron(pia,sg,output_registered_sticking_out_check+sg_filename+'.am',step_size=50)
    else:
        shutil.copyfile(output_corrected_spatial_graphs+sg_filename+'.am',\
                    output_registered_sticking_out_check+sg_filename+'.am')
    shutil.copyfile(output_surfaces+sg_filename+'_pia_bottom_open.vtk',\
                    output_registered_sticking_out_check+sg_filename+'_pia.vtk')
    shutil.copyfile(output_surfaces+sg_filename+'_WM_bottom_open.vtk',\
                    output_registered_sticking_out_check+sg_filename+'_WM.vtk')
    
    
    ################ Step 1###################################
    # Align barrels
    ##########################################################
    matching_barrels = MatchBarrels(ref_barrels_sg.barrels,sg.barrels,use_barrels_alone=True)
    ref_s1_pts,s1_pts = matching_barrels.get_matching_barrel_centroids()
    landmarks = Landmarks(pts=s1_pts)
    icp,txmat = landmarks.align_landmarks(ref_s1_pts)
    aligner = Alignment()
    #print(txmat)
    sg,sg_neuron,pia,wm,vM1_pia,vM1_wm = Alignment().transform_files(output_registered_sticking_out_check,output_registered_barrels,\
                                                  sg_filename,\
                            icp=icp,txmat = txmat,\
                            txfilename = output_registered_barrels+sg_filename+'_1_Barrel_Transformation_Matrix.txt')
    
    ################ Step 2###################################
    # Find global transformation using alignemnt method of choice
    ##########################################################
#     txmat = []
#     icp = vtk.vtkIterativeClosestPointTransform()

#     # align by entire pia wm surfaces
#     pia_wm = pia
#     pia.append(wm.surface)
#     aligner = Alignment(ref_pia_wm.surface,pia_wm.surface)
#     txmat = aligner.get_transformation_matrix()
#     icp = aligner.get_icp_transform()

#     sg,sg_neuron,pia,wm,vM1_pia,vM1_wm = Alignment().transform_files(output_registered_barrels,output_registered_global,sg_filename,\
#                             icp=icp,txmat = txmat,\
#                             txfilename = output_registered_global+sg_filename+'_2_Global_Transformation_Matrix.txt')

     
    ################ Step 2.5 ###################################
    # Clip to vM1 
    ##########################################################
    pia = Surface(output_registered_barrels+sg_filename+'_pia.vtk')
    wm = Surface(output_registered_barrels+sg_filename+'_WM.vtk')
    sg = AmiraSpatialGraph(output_registered_barrels+sg_filename+'.am')
    pcas,vm1_final,vM1_hull,vm1_pia,vm1_wm = Surface().get_vm1(pia.surface,wm.surface,\
                                            given_pcas=[ref_pca_0.graph_data.edge_list[0],\
                                                        ref_pca_1.graph_data.edge_list[0],\
                                                        ref_pca_2.graph_data.edge_list[0]],pts=[])


    Surface(polydata=vm1_pia).write_surface_mesh(output_registered_clipped_vm1+sg_filename+'_vM1_pia.vtk')
    Surface(polydata=vm1_wm).write_surface_mesh(output_registered_clipped_vm1+sg_filename+'_vM1_WM.vtk')
    sg.write_spatial_graph(output_registered_clipped_vm1+sg_filename+'.am')
    shutil.copyfile(output_registered_barrels+sg_filename+'_pia.vtk',output_registered_clipped_vm1+sg_filename+'_pia.vtk')
    shutil.copyfile(output_registered_barrels+sg_filename+'_WM.vtk',output_registered_clipped_vm1+sg_filename+'_WM.vtk')
     
    ################ Step 3 ###################################
    # Align to vM1
    ##########################################################
    vm1_pia = Surface(output_registered_clipped_vm1+sg_filename+'_vM1_pia.vtk')
    vm1_wm = Surface(output_registered_clipped_vm1+sg_filename+'_vM1_WM.vtk')
    vm1_pia_wm = vm1_pia
    vm1_pia_wm.append(vm1_wm.surface)

    aligner = Alignment(ref_vM1_pia_wm.surface,vm1_pia_wm.surface)
    txmat = aligner.get_transformation_matrix()
    icp = aligner.get_icp_transform()
    
    sg,sg_neuron,pia,wm,vM1_pia,vM1_wm = Alignment().transform_files(output_registered_clipped_vm1,output_registered_local,sg_filename,\
                            icp=icp,txmat = txmat,\
                            txfilename = output_registered_local+sg_filename+'_3_Global_Transformation_Matrix.txt')
    
    
    ################ Step 4 ###################################
    # fine tune to vM1 around neuron
    ##########################################################
    
    icp,txmat = vm1_local_alignment(sg,pia,ref_pia,number_of_pts=2,ref_pca=ref_pca_2)
    
    if len(txmat)==0:
        print('{} exp has problems'.format(sg_filename))
        return
    sg,sg_neuron,pia,wm,vM1_pia,vM1_wm = Alignment().transform_files(output_registered_local,output_registered_fine_tune_1,\
                                                                 sg_filename,icp=icp,txmat = txmat,\
                                                                 txfilename = output_registered_fine_tune_1\
                                                                 +sg_filename+'_3_1_Global_Transformation_Matrix.txt')

    #print(sg.neuron.dendrite.apical_dendrite.edge_list)
    icp,txmat = vm1_local_alignment(sg,pia,ref_pia,number_of_pts=1)
    if len(txmat)==0:
        print('{} exp has problems'.format(sg_filename))
        return
    sg,sg_neuron,pia,wm,vM1_pia,vM1_wm = Alignment().transform_files(output_registered_fine_tune_1,output_registered_fine_tune_2,\
                                                                 sg_filename,icp=icp,txmat = txmat,\
                                                                 txfilename = output_registered_fine_tune_2+\
                                                                 sg_filename+'_3_2_Global_Transformation_Matrix.txt')
    
     
    ################ Step 5 ###################################
    # Move to Center of vM1
    ##########################################################
    # Ref Axis: central axis of ref vm1
    # axis : apical vector - till intersects pia and wm
    ref_axis,neuron_axis = get_ref_and_neuron_axes(sg,pia,wm,ref_pia,ref_wm,ref_pca=ref_pca_2)
    if len(ref_axis)==0 or len(neuron_axis)==0:
        return
    aligner = Alignment(Surface(pts=ref_axis).surface,Surface(pts=neuron_axis).surface,mode='similarity')
    txmat = aligner.get_transformation_matrix()
    icp = aligner.get_icp_transform()
    sg_neuron.apply_transformation(txmat)
    sg_neuron.write_spatial_graph(output_registered_centered+sg_filename+'_reg_neuron.am')
    
    # Now unscale the neuron
    height = get_soma_to_apical_center_distance(sg)
    #print(height)
    soma_center,apical_center = get_soma_and_apical_centers(sg_neuron)
    if len(soma_center)==0 or len(apical_center)==0:
        return
    apical_target = Vectors().create_pt_along_vector_at_given_distance(height,soma_center,apical_center)
    #print(apical_target)
    soma_center_unscaled,apical_center_unscaled = get_soma_and_apical_centers(sg)
    if len(soma_center_unscaled)==0 or len(apical_center_unscaled)==0:
        return
    landmarks = Landmarks(pts=[soma_center_unscaled,apical_center_unscaled])
    icp,txmat = landmarks.align_landmarks([soma_center,apical_target])
    sg_neuron = AmiraSpatialGraph(output_registered_fine_tune_2+sg_filename+'_reg_neuron.am')
    sg_neuron.apply_transformation(txmat)
    sg_neuron.write_spatial_graph(output_registered_centered_unscaled+sg_filename+'_reg_neuron.am')
    
    ################ Step 6 ###################################
    # Transform to Local vM1 ref frame
    ##########################################################
    
    return  

In [ ]:
alignment_type = 'Using_vS1_vM1_Surface_Points'
fn_list_surfs = []
sg_files_list = sorted(glob.glob(output_corrected_spatial_graphs+'*.am'))
for i in range(len(sg_files_list)):
    sg_file = sg_files_list[i]
    if i == 8:
        print(i,sg_file)
        #fn_list_surfs.append(register_neurons(os.path.basename(sg_file)[:-3],alignment_type,check_neuron_sticking_out=False))
        register_neurons(os.path.basename(sg_file)[:-3],alignment_type,check_neuron_sticking_out=True)
#compute(fn_list_surfs,scheduler='multiprocessing')

# Merge Neurons for Visualization

In [ ]:
all_neurons = AmiraSpatialGraph(generic_graph=True)
for neuron_file in glob.glob(output_registered_fine_tune_2+'/*_reg_neuron.am'):
    sg_neuron = AmiraSpatialGraph(neuron_file)
    all_neurons.graph_data = all_neurons.combine_subgraphs([all_neurons.graph_data,sg_neuron.graph_data])

    
all_neurons.write_spatial_graph(output_root+'All_Neurons_Registered_To_vM1.am')


In [ ]:
all_neurons = AmiraSpatialGraph(generic_graph=True)
for neuron_file in glob.glob(output_registered_centered_unscaled+'/*_reg_neuron.am'):
    sg_neuron = AmiraSpatialGraph(neuron_file)
    all_neurons.graph_data = all_neurons.combine_subgraphs([all_neurons.graph_data,sg_neuron.graph_data])

    
all_neurons.write_spatial_graph(output_root+'All_Neurons_Registered_To_vM1_Central_Axis.am')

In [ ]:
all_neurons = AmiraSpatialGraph(generic_graph=True)
for neuron_file in glob.glob(output_registered_centered+'/*_reg_neuron.am'):
    sg_neuron = AmiraSpatialGraph(neuron_file)
    all_neurons.graph_data = all_neurons.combine_subgraphs([all_neurons.graph_data,sg_neuron.graph_data])

    
all_neurons.write_spatial_graph(output_root+'All_Neurons_Registered_To_vM1_Central_Axis_Scaled.am')